# Let’s use a simple example data to demonstrate how XGBoost algorithm works


### A) on a regression problem.

Let’s use a simple example data to demonstrate how XGBoost algorithm works on a regression problem.


#### Step 1. Set an initial prediction
Let’s set the initial prediction (F0(x)) to be 3. In other words, regardless of the value of X, the predicted Y will be 3 - mean value.


In [9]:
x = [5,8,15,20,-9]
y = [1,2,8,4,0]
pred_0 = sum(y) / len(y)
pred_0

3.0

### Step 2. Calculate the residuals
In this step we calculate the difference between the actual value (Y) and the predicted value (F0(x)).


In [13]:
res_0 = []
for i in range(len(y)):
    res_0.append(y[i] - pred_0)
res_0

[-2.0, -1.0, 5.0, 1.0, -3.0]

### Step 3. Train model on the residuals

Here, we build one or more tree-based models that are trained on [X, res].

The following shows how a tree model is constructed.

#### A) Generate split candidates ####

In this step we find one or more split candidates. The formula is with N rows we would have N - 1 split candidates.

With our dataset example, here’s the result.

The threshold will be used to create two child nodes. 

Here’s the complete parent-child nodes for each candidate.